# 1.Preparation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls "/content/drive/MyDrive/Your own address"

 Add_Clustered_news_0.6				     KO_2023-10-30_2024-09-30.json
'Benchmark with Llama3 8b Data.ipynb'		     MCD_2023-10-30_2024-09-30.json
 Clustered_news_0.6				    'News repetition reduction.ipynb'
 config.json					     __pycache__
 dataset_dict.json				     result.ipynb
 DIS_2023-10-30_2024-09-30.json			     test
 finetuned_models				     test_demo.ipynb
 fingpt-forecaster				    'teststing result.docx'
 fingpt-forecaster-1029				     train
 fingpt-forecaster-1102				     train_lora.py
 fingpt-forecaster-1105				     train.sh
 fingpt-forecaster-dow-30-20230930-20240930-1-4-06   Untitled
 fingpt-forecaster-dow-30-20231030-20240930-1-4-06   utils.py
 HD_2023-10-30_2024-09-30.json			     wandb
 HON_2023-10-30_2024-09-30.json


In [ ]:
%cd "/content/drive/MyDrive/Your own address"

/content/drive/MyDrive/Colab Notebooks/AI4Finance/FinForecaster/Benchmark with Llama3 8b Data


In [ ]:
from huggingface_hub import login


login(token="Your own key")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
!pip install transformers torch datasets peft wandb deepspeed rouge-score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 61.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.6 MB/s eta 0:00:00
  Created wheel for deepspeed: filename=deepspeed-0.15.4-py3-none-any.whl size=1527837 sha256=366dde8b5428c95919f649832ddbe912cd4f52bc187bd7dabb899d4c24eb28f3
  Stored in directory: /root/.cache/pi

In [ ]:
import os
import torch
import re



from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
from datasets import load_from_disk
from datasets import Dataset
from sklearn.metrics import accuracy_score
from tqdm import tqdm
# from peft import PeftModel
from utils import *



# 2. Load model and tokenizer

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    'meta-llama/Meta-Llama-3-8B',
    trust_remote_code=True,
    device_map="auto",
    torch_dtype=torch.float16,
)
base_model.model_parellal = True

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

In [ ]:
model = PeftModel.from_pretrained(base_model, 'The address where your parameters are saved after your training session is complete')
model_hg = PeftModel.from_pretrained(base_model, 'The address where your parameters are saved after your training session is complete')
# Note: If you use the code we provide, the training result will generally be automatically saved in the same folder as the training colab file.

model = model.eval()
model_hg = model_hg.eval()
base_model = base_model.eval()

In [ ]:

tokenizer = AutoTokenizer.from_pretrained('meta-llama/Meta-Llama-3-8B')


tokenizer.padding_side = "right"


tokenizer.pad_token_id = tokenizer.eos_token_id

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

# 3. Load the test set

In [ ]:

test_dataset = load_from_disk('/content/drive/MyDrive/Your test set address')

In [ ]:
test_dataset

Dataset({
    features: ['prompt', 'answer', 'period', 'label', 'symbol'],
    num_rows: 530
})

If you want to test on a company basis or want to see how the model predicts the stock price of different companies, you can use the following codes to filter the test set based on the stock code.

In [ ]:
def filter_by_ticker(test_dataset, ticker_code):

    filtered_data = []

    for row in test_dataset:
        prompt_content = row['prompt']

        ticker_symbol = re.search(r"ticker\s([A-Z]+)", prompt_content)

        if ticker_symbol and ticker_symbol.group(1) == ticker_code:
            filtered_data.append(row)

    filtered_dataset = Dataset.from_dict({key: [row[key] for row in filtered_data] for key in test_dataset.column_names})

    return filtered_dataset



In [ ]:
def get_unique_ticker_symbols(test_dataset):

    ticker_symbols = set()

    for i in range(len(test_dataset)):
        prompt_content = test_dataset[i]['prompt']

        ticker_symbol = re.search(r"ticker\s([A-Z]+)", prompt_content)

        if ticker_symbol:
            ticker_symbols.add(ticker_symbol.group(1))

    return list(ticker_symbols)



def insert_guidance_after_intro(prompt):

    intro_marker = (
        "[INST]<<SYS>>\n"
        "You are a seasoned stock market analyst. Your task is to list the positive developments and "
        "potential concerns for companies based on relevant news and basic financials from the past weeks, "
        "then provide an analysis and prediction for the companies' stock price movement for the upcoming week."
    )
    guidance_start_marker = "Based on all the information before"
    guidance_end_marker = "Following these instructions, please come up with 2-4 most important positive factors"

    intro_pos = prompt.find(intro_marker)
    guidance_start_pos = prompt.find(guidance_start_marker)
    guidance_end_pos = prompt.find(guidance_end_marker)

    if intro_pos == -1 or guidance_start_pos == -1 or guidance_end_pos == -1:
        return prompt

    guidance_section = prompt[guidance_start_pos:guidance_end_pos].strip()

    new_prompt = (
        f"{prompt[:intro_pos + len(intro_marker)]}\n\n"
        f"{guidance_section}\n\n"
        f"{prompt[intro_pos + len(intro_marker):guidance_start_pos]}"
        f"{prompt[guidance_end_pos:]}"
    )

    return new_prompt


def apply_to_all_prompts_in_dataset(test_dataset):

    updated_dataset = test_dataset.map(lambda x: {"prompt": insert_guidance_after_intro(x["prompt"])})

    return updated_dataset

In [ ]:
test_dataset = apply_to_all_prompts_in_dataset(test_dataset)

In [ ]:

unique_symbols = set(test_dataset['symbol'])

print(f"Total number of unique stock codes: {len(unique_symbols)}")

print(f"Unique stock codes: {unique_symbols}")


Total number of unique stock codes: 30
Unique stock codes: {'DIS', 'WMT', 'TRV', 'KO', 'CSCO', 'GS', 'AXP', 'MCD', 'CRM', 'MSFT', 'JNJ', 'MRK', 'PG', 'JPM', 'DOW', 'HON', 'VZ', 'CAT', 'UNH', 'BA', 'NKE', 'INTC', 'AMGN', 'CVX', 'MMM', 'AAPL', 'IBM', 'HD', 'WBA', 'V'}


In [ ]:
test_dataset_AXP['symbol'][0]

'AXP'

In [ ]:
test_dataset_AAPL = filter_by_ticker(test_dataset, "AAPL")
print(test_dataset_AAPL)

Dataset({
    features: ['prompt', 'answer', 'period', 'label', 'symbol'],
    num_rows: 19
})


In [ ]:
test_dataset_AMGN = filter_by_ticker(test_dataset, "AMGN")
print(test_dataset_AMGN)

Dataset({
    features: ['prompt', 'answer', 'period', 'label', 'symbol'],
    num_rows: 19
})


In [ ]:
test_dataset_AMGN['prompt'][0]

"[INST]<<SYS>>\nYou are a seasoned stock market analyst. Your task is to list the positive developments and potential concerns for companies based on relevant news and basic financials from the past weeks, then provide an analysis and prediction for the companies' stock price movement for the upcoming week. Your answer format should be as follows:\n\n[Positive Developments]:\n1. ...\n\n[Potential Concerns]:\n1. ...\n\n[Prediction & Analysis]\nPrediction: ...\nAnalysis: ...\n\n<</SYS>>\n\n[Company Introduction]:\n\nAmgen Inc is a leading entity in the Biotechnology sector. Incorporated and publicly traded since 1983-06-17, the company has established its reputation as one of the key players in the market. As of today, Amgen Inc has a market capitalization of 171526.24 in USD, with 537.33 shares outstanding.\n\nAmgen Inc operates primarily in the US, trading under the ticker AMGN on the NASDAQ NMS - GLOBAL MARKET. As a dominant force in the Biotechnology space, the company continues to i

In [ ]:
test_dataset_AXP_new = apply_to_all_prompts_in_dataset(test_dataset_AXP)

Map:   0%|          | 0/19 [00:00<?, ? examples/s]

In [ ]:
test_dataset_AAPL_new = apply_to_all_prompts_in_dataset(test_dataset_AAPL)

Map:   0%|          | 0/19 [00:00<?, ? examples/s]

In [ ]:
test_dataset_AMGN_new = apply_to_all_prompts_in_dataset(test_dataset_AMGN)

Map:   0%|          | 0/19 [00:00<?, ? examples/s]

In [ ]:
test_dataset_AMGN_new['prompt'][0]

"[INST]<<SYS>>\nYou are a seasoned stock market analyst. Your task is to list the positive developments and potential concerns for companies based on relevant news and basic financials from the past weeks, then provide an analysis and prediction for the companies' stock price movement for the upcoming week.\n\nBased on all the information before 2024-05-26 and the following guidelines, let's analyze the positive developments and potential concerns for AAPL. Apart from stock price information and basic financials, several news topcis are given and they are derived from news clustering. Each topic contains its duration, a key news headline and summary, and a topic size, indicating the number of related news items within that cluster. First, note that some of the factors only influence the price for the following 1 or 2 days while some others may have longer-term effects. Second, the topics that are closer to 2023-11-12 are likely to have a stronger influence on the upcoming stock price f

# 4.Testing

In [ ]:
def test_demo(model, tokenizer, prompt):

    inputs = tokenizer(
        prompt, return_tensors='pt',
        padding=False, max_length=8000
    )
    inputs = {key: value.to(model.device) for key, value in inputs.items()}

    res = model.generate(
        **inputs, max_length=4096, do_sample=True,
        eos_token_id=tokenizer.eos_token_id,
        use_cache=True
    )
    output = tokenizer.decode(res[0], skip_special_tokens=True)
    return output
    # return res

In [ ]:
from sklearn.metrics import accuracy_score
import re
from tqdm import tqdm


answers_base, answers_fine_tuned, gts = [], [], []

for i in tqdm(range(len(test_dataset)), desc="Processing test samples"):
    try:
        prompt = test_dataset[i]['prompt']
        gt = test_dataset[i]['answer']

        output_base = test_demo(base_model, tokenizer, prompt)
        answer_base = re.sub(r'.*\[/INST\]\s*', '', output_base, flags=re.DOTALL)

        output_fine_tuned = test_demo(model, tokenizer, prompt)
        answer_fine_tuned = re.sub(r'.*\[/INST\]\s*', '', output_fine_tuned, flags=re.DOTALL)

        answers_base.append(answer_base)
        answers_fine_tuned.append(answer_fine_tuned)
        gts.append(gt)

    except Exception as e:
        print(f"Error processing sample {i}: {e}")

Processing test samples:   0%|          | 0/570 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Processing test samples:   0%|          | 0/570 [00:13<?, ?it/s]


KeyboardInterrupt: 

This function is used to evaluate the predictive accuracy of a model for all data in the test set.

In [ ]:
def test_acc(test_dataset):
  answers_base, answers_fine_tuned, gts = [], [], []

  for i in tqdm(range(len(test_dataset)), desc="Processing test samples"):
      try:
          prompt = test_dataset[i]['prompt']
          gt = test_dataset[i]['answer']

          output_base = test_demo(base_model, tokenizer, prompt)
          answer_base = re.sub(r'.*\[/INST\]\s*', '', output_base, flags=re.DOTALL)

          output_fine_tuned = test_demo(model, tokenizer, prompt)
          answer_fine_tuned = re.sub(r'.*\[/INST\]\s*', '', output_fine_tuned, flags=re.DOTALL)

          answers_base.append(answer_base)
          answers_fine_tuned.append(answer_fine_tuned)
          gts.append(gt)

      except Exception as e:
          print(f"Error processing sample {i}: {e}")
  return answers_base, answers_fine_tuned,gts

This function is used to evaluate the model's prediction results for data from a specific company (symbol).

In [ ]:
def calculate_acc_per_symbol(test_dataset, base_model, fine_tuned_model, tokenizer):
    symbol_metrics = {}
    unique_symbols = set(test_dataset['symbol'])


    for symbol in unique_symbols:
        print(f"Processing symbol: {symbol}")


        symbol_data = test_dataset.filter(lambda x: x['symbol'] == symbol)

        answers_base, answers_fine_tuned, gts = [], [], []


        for i in tqdm(range(len(symbol_data)), desc=f"Processing samples for {symbol}"):
            try:
                prompt = symbol_data[i]['prompt']
                gt = symbol_data[i]['answer']


                output_base = test_demo(base_model, tokenizer, prompt)
                answer_base = re.sub(r'.*\[/INST\]\s*', '', output_base, flags=re.DOTALL)

                output_fine_tuned = test_demo(fine_tuned_model, tokenizer, prompt)
                answer_fine_tuned = re.sub(r'.*\[/INST\]\s*', '', output_fine_tuned, flags=re.DOTALL)

                answers_base.append(answer_base)
                answers_fine_tuned.append(answer_fine_tuned)
                gts.append(gt)

            except Exception as e:
                print(f"Error processing sample {i} for symbol {symbol}: {e}")


        metrics_base = calc_metrics(answers_base, gts)
        metrics_fine_tuned = calc_metrics(answers_fine_tuned, gts)


        symbol_metrics[symbol] = {
            "base_model_metrics": metrics_base,
            "fine_tuned_model_metrics": metrics_fine_tuned
        }

    return symbol_metrics


AMD:

Binary Accuracy: 0.50  |  Mean Square Error: 39.61

Rouge Score of Positive Developments: {'rouge1': 0.5060177553893659, 'rouge2': 0.1741811068503716, 'rougeL': 0.2980045119994417}

Rouge Score of Potential Concerns: {'rouge1': 0.4320301010968262, 'rouge2': 0.11705631429903035, 'rougeL': 0.23476935143404398}

Rouge Score of Summary Analysis: {'rouge1': 0.470459778622912, 'rouge2': 0.14549648189945077, 'rougeL': 0.23199012435457333}

Binary Accuracy: 0.56  |  Mean Square Error: 36.89

Rouge Score of Positive Developments: {'rouge1': 0.5013257403701984, 'rouge2': 0.17160271287637208, 'rougeL': 0.2765109599822412}

Rouge Score of Potential Concerns: {'rouge1': 0.42679645776577774, 'rouge2': 0.11955874122187964, 'rougeL': 0.2439839872312159}

Rouge Score of Summary Analysis: {'rouge1': 0.49239721054961194, 'rouge2': 0.1491389887533065, 'rougeL': 0.24172794173827286}



TSLA:

Binary Accuracy: 0.53  |  Mean Square Error: 23.89

Rouge Score of Positive Developments: {'rouge1': 0.4955854560308768, 'rouge2': 0.18580716071155987, 'rougeL': 0.30459609561463113}

Rouge Score of Potential Concerns: {'rouge1': 0.45721507013860124, 'rouge2': 0.16883641324246051, 'rougeL': 0.28556722767739756}

Rouge Score of Summary Analysis: {'rouge1': 0.4691666536421016, 'rouge2': 0.13760697116145462, 'rougeL': 0.2349413533316067}

Binary Accuracy: 0.58  |  Mean Square Error: 22.58

Rouge Score of Positive Developments: {'rouge1': 0.4758367091365868, 'rouge2': 0.16566794148490854, 'rougeL': 0.2857270432003906}

Rouge Score of Potential Concerns: {'rouge1': 0.44977872673340447, 'rouge2': 0.16485333744699074, 'rougeL': 0.29809084649515283}

Rouge Score of Summary Analysis: {'rouge1': 0.47395686964563516, 'rouge2': 0.1393596974109519, 'rougeL': 0.23429647472672885}


CEG:

Binary Accuracy: 0.56  |  Mean Square Error: 42.44

Rouge Score of Positive Developments: {'rouge1': 0.5108622631717826, 'rouge2': 0.21790882997716549, 'rougeL': 0.32972183999689525}

Rouge Score of Potential Concerns: {'rouge1': 0.42048202209863894, 'rouge2': 0.12751400507216892, 'rougeL': 0.24934096282510254}

Rouge Score of Summary Analysis: {'rouge1': 0.4583018775461255, 'rouge2': 0.1401242211215103, 'rougeL': 0.22690845487821173}

Binary Accuracy: 0.59  |  Mean Square Error: 36.94

Rouge Score of Positive Developments: {'rouge1': 0.5235472528020955, 'rouge2': 0.2005736510145733, 'rougeL': 0.31953434551717946}

Rouge Score of Potential Concerns: {'rouge1': 0.4525749607991249, 'rouge2': 0.13527392723109263, 'rougeL': 0.26175262148128187}

Rouge Score of Summary Analysis: {'rouge1': 0.4886748580080039, 'rouge2': 0.15530722436934705, 'rougeL': 0.236985159361098}



CRWD:

Binary Accuracy: 0.50  |  Mean Square Error: 41.67

Rouge Score of Positive Developments: {'rouge1': 0.48155046619540876, 'rouge2': 0.16640213285548217, 'rougeL': 0.2992946369735689}

Rouge Score of Potential Concerns: {'rouge1': 0.4328355113217057, 'rouge2': 0.14135312190678792, 'rougeL': 0.2775081856613268}

Rouge Score of Summary Analysis: {'rouge1': 0.4728548982525917, 'rouge2': 0.14080500004119317, 'rougeL': 0.23896424439091943}

Binary Accuracy: 0.47  |  Mean Square Error: 39.95

Rouge Score of Positive Developments: {'rouge1': 0.4727759985491417, 'rouge2': 0.17148213489822542, 'rougeL': 0.30628217430087534}

Rouge Score of Potential Concerns: {'rouge1': 0.4467224696542352, 'rouge2': 0.14804014578465055, 'rougeL': 0.27756822386985275}

Rouge Score of Summary Analysis: {'rouge1': 0.48301158845346737, 'rouge2': 0.15113079450489178, 'rougeL': 0.23670955346412748}


XEL:

Binary Accuracy: 0.56  |  Mean Square Error: 4.89

Rouge Score of Positive Developments: {'rouge1': 0.5064365480332333, 'rouge2': 0.19608131212548877, 'rougeL': 0.3070568765978633}

Rouge Score of Potential Concerns: {'rouge1': 0.4109001753037815, 'rouge2': 0.11984506983524759, 'rougeL': 0.26367335406795406}

Rouge Score of Summary Analysis: {'rouge1': 0.46377798059585285, 'rouge2': 0.13651898628351553, 'rougeL': 0.2466859671025123}

Binary Accuracy: 0.74  |  Mean Square Error: 4.84

Rouge Score of Positive Developments: {'rouge1': 0.4928056499614663, 'rouge2': 0.18805595912689962, 'rougeL': 0.30947819127901494}

Rouge Score of Potential Concerns: {'rouge1': 0.40990241297484037, 'rouge2': 0.12655796340799857, 'rougeL': 0.24277686419790556}

Rouge Score of Summary Analysis: {'rouge1': 0.48574559901416486, 'rouge2': 0.15003926087079708, 'rougeL': 0.2444016392208824}


VST:

Binary Accuracy: 0.61  |  Mean Square Error: 22.83

Rouge Score of Positive Developments: {'rouge1': 0.5192480372889094, 'rouge2': 0.19681931999911356, 'rougeL': 0.31764684034235224}

Rouge Score of Potential Concerns: {'rouge1': 0.43365789936239474, 'rouge2': 0.1364632945334866, 'rougeL': 0.25635318043876015}

Rouge Score of Summary Analysis: {'rouge1': 0.4806898829238357, 'rouge2': 0.1408750143948886, 'rougeL': 0.23431468225791935}

Binary Accuracy: 0.59  |  Mean Square Error: 35.24

Rouge Score of Positive Developments: {'rouge1': 0.5198402415499522, 'rouge2': 0.1994169563022763, 'rougeL': 0.3033667887470872}

Rouge Score of Potential Concerns: {'rouge1': 0.44289668743813726, 'rouge2': 0.14372259756758426, 'rougeL': 0.2752725647956803}

Rouge Score of Summary Analysis: {'rouge1': 0.4748963937159137, 'rouge2': 0.14385664082196298, 'rougeL': 0.23608888544912704}


GOOG:

Binary Accuracy: 0.53  |  Mean Square Error: 11.68

Rouge Score of Positive Developments: {'rouge1': 0.4840547357914029, 'rouge2': 0.16926713317343478, 'rougeL': 0.29431028379721835}

Rouge Score of Potential Concerns: {'rouge1': 0.44005460980013716, 'rouge2': 0.1413277244534965, 'rougeL': 0.270330232317154}

Rouge Score of Summary Analysis: {'rouge1': 0.47461510368805493, 'rouge2': 0.14169379373024102, 'rougeL': 0.24057318912931852}

Binary Accuracy: 0.61  |  Mean Square Error: 10.50

Rouge Score of Positive Developments: {'rouge1': 0.49731909117373324, 'rouge2': 0.17842148203281083, 'rougeL': 0.3131729542543711}

Rouge Score of Potential Concerns: {'rouge1': 0.4422309289628772, 'rouge2': 0.15629771070656226, 'rougeL': 0.2797110771397098}

Rouge Score of Summary Analysis: {'rouge1': 0.45587751891005257, 'rouge2': 0.13374550432378426, 'rougeL': 0.22164697541439782}



AAPL:

Binary Accuracy: 0.79  |  Mean Square Error: 18.11

Rouge Score of Positive Developments: {'rouge1': 0.5074943278107572, 'rouge2': 0.1795394134223658, 'rougeL': 0.3047268587330497}

Rouge Score of Potential Concerns: {'rouge1': 0.4700635118741938, 'rouge2': 0.1514768706432656, 'rougeL': 0.2787390782259439}

Rouge Score of Summary Analysis: {'rouge1': 0.4615849531647695, 'rouge2': 0.14103830669059378, 'rougeL': 0.23395830867170173}

Binary Accuracy: 0.63  |  Mean Square Error: 19.95

Rouge Score of Positive Developments: {'rouge1': 0.5113683307401696, 'rouge2': 0.1966516233909619, 'rougeL': 0.3186625206074603}

Rouge Score of Potential Concerns: {'rouge1': 0.4813836391347546, 'rouge2': 0.16275360313031614, 'rougeL': 0.29822886076960264}

Rouge Score of Summary Analysis: {'rouge1': 0.46538859569893193, 'rouge2': 0.13071417024354628, 'rougeL': 0.23022148689089084}


VZ:

Binary Accuracy: 0.63  |  Mean Square Error: 8.21

Rouge Score of Positive Developments: {'rouge1': 0.5541870488132258, 'rouge2': 0.24687024062470897, 'rougeL': 0.38322924113978335}

Rouge Score of Potential Concerns: {'rouge1': 0.4191479511153993, 'rouge2': 0.1335303509781738, 'rougeL': 0.25903057741311164}

Rouge Score of Summary Analysis: {'rouge1': 0.47521650171098184, 'rouge2': 0.14920375476411704, 'rougeL': 0.24864026246778095}

Binary Accuracy: 0.61  |  Mean Square Error: 8.22

Rouge Score of Positive Developments: {'rouge1': 0.5360594178134275, 'rouge2': 0.21601113054296522, 'rougeL': 0.3349334592033867}

Rouge Score of Potential Concerns: {'rouge1': 0.4348214942419463, 'rouge2': 0.14137369184726012, 'rougeL': 0.28181909577906583}

Rouge Score of Summary Analysis: {'rouge1': 0.4602646401045971, 'rouge2': 0.13272675334469491, 'rougeL': 0.2197110031245989}


INTC:


Binary Accuracy: 0.42  |  Mean Square Error: 34.47

Rouge Score of Positive Developments: {'rouge1': 0.5044366137949622, 'rouge2': 0.19472265558359955, 'rougeL': 0.3168994473772751}

Rouge Score of Potential Concerns: {'rouge1': 0.4384399454673387, 'rouge2': 0.12502801663887333, 'rougeL': 0.2579521552333636}

Rouge Score of Summary Analysis: {'rouge1': 0.44826143394766843, 'rouge2': 0.12930680885241969, 'rougeL': 0.2295481159513373}

Binary Accuracy: 0.42  |  Mean Square Error: 37.53

Rouge Score of Positive Developments: {'rouge1': 0.5123696956778591, 'rouge2': 0.1834028806034139, 'rougeL': 0.3011073547976688}

Rouge Score of Potential Concerns: {'rouge1': 0.43267917522601007, 'rouge2': 0.1231402962933088, 'rougeL': 0.2533042863123596}

Rouge Score of Summary Analysis: {'rouge1': 0.46644929098514637, 'rouge2': 0.13671058812005013, 'rougeL': 0.22050506934872763}


AVGO:

Binary Accuracy: 0.39  |  Mean Square Error: 39.39

Rouge Score of Positive Developments: {'rouge1': 0.5051663103320295, 'rouge2': 0.1954931131036871, 'rougeL': 0.31404490188844253}

Rouge Score of Potential Concerns: {'rouge1': 0.4034919199773857, 'rouge2': 0.11701674800201098, 'rougeL': 0.24856414863790516}

Rouge Score of Summary Analysis: {'rouge1': 0.4855187933796391, 'rouge2': 0.15018704129157406, 'rougeL': 0.2384208014868121}

Binary Accuracy: 0.42  |  Mean Square Error: 31.00

Rouge Score of Positive Developments: {'rouge1': 0.5161911854377392, 'rouge2': 0.19329678807852307, 'rougeL': 0.3174628734723166}

Rouge Score of Potential Concerns: {'rouge1': 0.3898818438226846, 'rouge2': 0.09656287647209671, 'rougeL': 0.20843759726729894}

Rouge Score of Summary Analysis: {'rouge1': 0.47232512718053765, 'rouge2': 0.13426963230815145, 'rougeL': 0.23757761368631763}


BA:

Binary Accuracy: 0.44  |  Mean Square Error: 30.81

Rouge Score of Positive Developments: {'rouge1': 0.5035205412609165, 'rouge2': 0.22604483110252127, 'rougeL': 0.34739719752389064}

Rouge Score of Potential Concerns: {'rouge1': 0.47292023886176965, 'rouge2': 0.16013181111462055, 'rougeL': 0.27682596956838246}

Rouge Score of Summary Analysis: {'rouge1': 0.450436613320355, 'rouge2': 0.13184015629346243, 'rougeL': 0.23837404695384337}

Binary Accuracy: 0.62  |  Mean Square Error: 21.31

Rouge Score of Positive Developments: {'rouge1': 0.49224906197048685, 'rouge2': 0.2046966846977495, 'rougeL': 0.32660609913299354}

Rouge Score of Potential Concerns: {'rouge1': 0.47667496340605314, 'rouge2': 0.16941686801384065, 'rougeL': 0.29870395765383967}

Rouge Score of Summary Analysis: {'rouge1': 0.4586082863311141, 'rouge2': 0.12989634871606792, 'rougeL': 0.24085422436245105}


VKTX:

Binary Accuracy: 0.56  |  Mean Square Error: 32.00

Rouge Score of Positive Developments: {'rouge1': 0.5146670548959649, 'rouge2': 0.23075689193195392, 'rougeL': 0.3288873142784038}

Rouge Score of Potential Concerns: {'rouge1': 0.4370243345723306, 'rouge2': 0.14456637347580437, 'rougeL': 0.2851113631065537}

Rouge Score of Summary Analysis: {'rouge1': 0.4670337874961807, 'rouge2': 0.14326068992483815, 'rougeL': 0.2350062437595016}

Binary Accuracy: 0.58  |  Mean Square Error: 36.68

Rouge Score of Positive Developments: {'rouge1': 0.5245794862742773, 'rouge2': 0.2285185805821582, 'rougeL': 0.3369880670987263}

Rouge Score of Potential Concerns: {'rouge1': 0.44811422400364676, 'rouge2': 0.1567177569186717, 'rougeL': 0.2798021796877333}

Rouge Score of Summary Analysis: {'rouge1': 0.4680272987539452, 'rouge2': 0.14691618052617733, 'rougeL': 0.23678393774028422}


META:

Binary Accuracy: 0.50  |  Mean Square Error: 22.56

Rouge Score of Positive Developments: {'rouge1': 0.49846719446047466, 'rouge2': 0.18695011373342413, 'rougeL': 0.32505385544325277}

Rouge Score of Potential Concerns: {'rouge1': 0.44546438318853127, 'rouge2': 0.13179367009196388, 'rougeL': 0.27606607959104484}

Rouge Score of Summary Analysis: {'rouge1': 0.45663704938185723, 'rouge2': 0.13272638374896073, 'rougeL': 0.22000374975851872}

Binary Accuracy: 0.44  |  Mean Square Error: 20.28

Rouge Score of Positive Developments: {'rouge1': 0.4844332833267155, 'rouge2': 0.1803067710928415, 'rougeL': 0.28603763827249734}

Rouge Score of Potential Concerns: {'rouge1': 0.44343720240159956, 'rouge2': 0.13335975040567682, 'rougeL': 0.2757697250960148}

Rouge Score of Summary Analysis: {'rouge1': 0.4704693226031723, 'rouge2': 0.13293609397051875, 'rougeL': 0.22312106297377088}

MSFT:

Binary Accuracy: 0.53  |  Mean Square Error: 14.21

Rouge Score of Positive Developments: {'rouge1': 0.4621148909857341, 'rouge2': 0.1338904227199656, 'rougeL': 0.264751121313342}

Rouge Score of Potential Concerns: {'rouge1': 0.4560937621301571, 'rouge2': 0.1437937493137033, 'rougeL': 0.2869816972862205}

Rouge Score of Summary Analysis: {'rouge1': 0.4682622751676134, 'rouge2': 0.14248295158646, 'rougeL': 0.22647684003418467}

Binary Accuracy: 0.47  |  Mean Square Error: 23.32

Rouge Score of Positive Developments: {'rouge1': 0.489691890864076, 'rouge2': 0.16823755566963342, 'rougeL': 0.29298327890533055}

Rouge Score of Potential Concerns: {'rouge1': 0.45764808313262845, 'rouge2': 0.14415347752570182, 'rougeL': 0.2705659197016818}

Rouge Score of Summary Analysis: {'rouge1': 0.4628886163042538, 'rouge2': 0.1342030970432304, 'rougeL': 0.2248523901142645}

MU:


Binary Accuracy: 0.35  |  Mean Square Error: 44.35

Rouge Score of Positive Developments: {'rouge1': 0.5163402321186589, 'rouge2': 0.18722459797175894, 'rougeL': 0.323849489256717}

Rouge Score of Potential Concerns: {'rouge1': 0.4422813987886004, 'rouge2': 0.12595143521221225, 'rougeL': 0.2771466455786268}

Rouge Score of Summary Analysis: {'rouge1': 0.46706410901019557, 'rouge2': 0.13328113859337776, 'rougeL': 0.22406268422073966}

Binary Accuracy: 0.56  |  Mean Square Error: 35.44

Rouge Score of Positive Developments: {'rouge1': 0.5014003131243928, 'rouge2': 0.19306891913194849, 'rougeL': 0.31649364611368047}

Rouge Score of Potential Concerns: {'rouge1': 0.42695391163884566, 'rouge2': 0.12786531551459535, 'rougeL': 0.25611624346046086}

Rouge Score of Summary Analysis: {'rouge1': 0.4770436933450599, 'rouge2': 0.13693796106748296, 'rougeL': 0.2290923200620776}


ZTS:

Binary Accuracy: 0.68  |  Mean Square Error: 6.58

Rouge Score of Positive Developments: {'rouge1': 0.5280181386829949, 'rouge2': 0.20774438211628393, 'rougeL': 0.32154185270560687}

Rouge Score of Potential Concerns: {'rouge1': 0.41529316328980187, 'rouge2': 0.12819144473828895, 'rougeL': 0.2695939146302464}

Rouge Score of Summary Analysis: {'rouge1': 0.4592058663827326, 'rouge2': 0.1319873531137652, 'rougeL': 0.23848798470110574}

Binary Accuracy: 0.59  |  Mean Square Error: 10.06

Rouge Score of Positive Developments: {'rouge1': 0.49493383304925753, 'rouge2': 0.18134200624860597, 'rougeL': 0.3192564704446295}

Rouge Score of Potential Concerns: {'rouge1': 0.40011403612586605, 'rouge2': 0.11489242051392032, 'rougeL': 0.25466683748879787}

Rouge Score of Summary Analysis: {'rouge1': 0.45014368744271793, 'rouge2': 0.13159929805395776, 'rougeL': 0.2307819283609618}

SHW:

Binary Accuracy: 0.53  |  Mean Square Error: 13.16

Rouge Score of Positive Developments: {'rouge1': 0.5271904983502017, 'rouge2': 0.20949277077425862, 'rougeL': 0.3184654542302742}

Rouge Score of Potential Concerns: {'rouge1': 0.4561185789055657, 'rouge2': 0.1464359095605757, 'rougeL': 0.26399484738722806}

Rouge Score of Summary Analysis: {'rouge1': 0.4654701407966681, 'rouge2': 0.14201454951571524, 'rougeL': 0.24597224236317397}

Binary Accuracy: 0.68  |  Mean Square Error: 9.26

Rouge Score of Positive Developments: {'rouge1': 0.5390541710617475, 'rouge2': 0.2202892599278583, 'rougeL': 0.3299451360611247}

Rouge Score of Potential Concerns: {'rouge1': 0.41406470805593903, 'rouge2': 0.11649196546264166, 'rougeL': 0.24652163227644017}

Rouge Score of Summary Analysis: {'rouge1': 0.4616846923767754, 'rouge2': 0.13528897366998457, 'rougeL': 0.23565409873684234}

In [ ]:
symbol_metrics

In [ ]:
answers_base_amgn, answers_fine_tuned_amgn,gts_amgn = test_acc(test_dataset_AMGN)

Processing test samples:   0%|          | 0/19 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Processing test samples:   5%|▌         | 1/19 [00:41<12:27, 41.51s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Processing test samples:  11%|█         | 2/19 [01:43<15:09, 53.50s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Processing test samples:  16%|█▌        | 3/19 [02:36<14:10, 53.15s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Processing test samples:  21%|██        | 4/19 [03:30<13:26, 53.75s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generati

# 5.Print test results

Here, three individual company datasets are used as examples. Note that the datasets used here are individual company datasets filtered using the filter function defined above. The function used to calculate the accuracy is the cacl_metrics function in utils.py. Another option is to calculate the accuracy using the acc calculation function defined above.

In [ ]:

print("\nEvaluating Base Model...")
base_metrics = calc_metrics(answers_base_axp, gts_axp)

print("\nEvaluating Fine-Tuned Model...")
fine_tuned_metrics = calc_metrics(answers_fine_tuned_axp, gts_axp)

print("\nBase Model Metrics:")
print(base_metrics)

print("\nFine-Tuned Model Metrics:")
print(fine_tuned_metrics)


Evaluating Base Model...

Binary Accuracy: 0.58  |  Mean Square Error: 10.53

Rouge Score of Positive Developments: {'rouge1': 0.5223651544398836, 'rouge2': 0.21362713277355666, 'rougeL': 0.33522895176684636}

Rouge Score of Potential Concerns: {'rouge1': 0.4023204504978999, 'rouge2': 0.12352515463564404, 'rougeL': 0.25194228304506106}

Rouge Score of Summary Analysis: {'rouge1': 0.46768523659443323, 'rouge2': 0.1374051348274996, 'rougeL': 0.23872450947508786}

Evaluating Fine-Tuned Model...

Binary Accuracy: 0.44  |  Mean Square Error: 10.22

Rouge Score of Positive Developments: {'rouge1': 0.5290888885650544, 'rouge2': 0.21237418298672206, 'rougeL': 0.31349996325313795}

Rouge Score of Potential Concerns: {'rouge1': 0.42372475951402305, 'rouge2': 0.11340730611523961, 'rougeL': 0.24092155525212838}

Rouge Score of Summary Analysis: {'rouge1': 0.4679287192335615, 'rouge2': 0.1287125175625126, 'rougeL': 0.22998802901443488}

Base Model Metrics:
{'valid_count': 19, 'bin_acc': 0.57894736

In [ ]:

print("\nEvaluating Base Model...")
base_metrics = calc_metrics(answers_base_aapl, gts_aapl)


print("\nEvaluating Fine-Tuned Model...")
fine_tuned_metrics = calc_metrics(answers_fine_tuned_aapl, gts_aapl)


print("\nBase Model Metrics:")
print(base_metrics)

print("\nFine-Tuned Model Metrics:")
print(fine_tuned_metrics)


Evaluating Base Model...

Binary Accuracy: 0.67  |  Mean Square Error: 7.50

Rouge Score of Positive Developments: {'rouge1': 0.5187885998413467, 'rouge2': 0.19345871742691625, 'rougeL': 0.30822545125472245}

Rouge Score of Potential Concerns: {'rouge1': 0.45678910561605257, 'rouge2': 0.1333132694798399, 'rougeL': 0.2718619761770612}

Rouge Score of Summary Analysis: {'rouge1': 0.47424456943014565, 'rouge2': 0.13376430906098646, 'rougeL': 0.24162801263470104}

Evaluating Fine-Tuned Model...

Binary Accuracy: 0.42  |  Mean Square Error: 10.42

Rouge Score of Positive Developments: {'rouge1': 0.5157843287373985, 'rouge2': 0.18869514320087907, 'rougeL': 0.3051289829326338}

Rouge Score of Potential Concerns: {'rouge1': 0.4723498461242249, 'rouge2': 0.17268287733224974, 'rougeL': 0.30508010786916423}

Rouge Score of Summary Analysis: {'rouge1': 0.4878082023020895, 'rouge2': 0.15133388453068694, 'rougeL': 0.25018007882963167}

Base Model Metrics:
{'valid_count': 18, 'bin_acc': 0.6666666666

In [ ]:

print("\nEvaluating Base Model...")
base_metrics = calc_metrics(answers_base_amgn, gts_amgn)


print("\nEvaluating Fine-Tuned Model...")
fine_tuned_metrics = calc_metrics(answers_fine_tuned_amgn, gts_amgn)


print("\nBase Model Metrics:")
print(base_metrics)

print("\nFine-Tuned Model Metrics:")
print(fine_tuned_metrics)


Evaluating Base Model...

Binary Accuracy: 0.32  |  Mean Square Error: 9.00

Rouge Score of Positive Developments: {'rouge1': 0.5059111306911762, 'rouge2': 0.17802353452066627, 'rougeL': 0.3149391449609365}

Rouge Score of Potential Concerns: {'rouge1': 0.40544265622054804, 'rouge2': 0.10849802495563123, 'rougeL': 0.25282588218903584}

Rouge Score of Summary Analysis: {'rouge1': 0.4572657735804972, 'rouge2': 0.1323204585199274, 'rougeL': 0.2435827300185122}

Evaluating Fine-Tuned Model...

Binary Accuracy: 0.32  |  Mean Square Error: 9.42

Rouge Score of Positive Developments: {'rouge1': 0.49511856577750796, 'rouge2': 0.1501482456670449, 'rougeL': 0.28433235831688647}

Rouge Score of Potential Concerns: {'rouge1': 0.4183051643018712, 'rouge2': 0.1321117418364849, 'rougeL': 0.24975751310768762}

Rouge Score of Summary Analysis: {'rouge1': 0.46229185132370393, 'rouge2': 0.12473982348098471, 'rougeL': 0.24004280145405746}

Base Model Metrics:
{'valid_count': 19, 'bin_acc': 0.315789473684